In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteUAHC2P/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteUAHC2P/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteUAHC2P/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [3]:
tokenizer = info.features['text'].encoder
imdb_train = imdb['train'].padded_batch(256)
imdb_test = imdb['test'].padded_batch(256)

In [4]:
lstm_size = 64
vocab_size = tokenizer.vocab_size
embedding_dim = 64
dense_dim = 64
epochs=10

In [5]:
import tensorflow.keras as keras

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_size, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_size, return_sequences=False)),
    keras.layers.Dense(dense_dim, activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 697025 (2.66 MB)
Trainable params: 697025 

In [8]:
#hostory = model.fit(imdb_train, epochs=epochs, validation_data=imdb_test)
history = model.fit(imdb_train, epochs=epochs, validation_data=imdb_test)

Epoch 1/10
98/98 [==============================] - 915s 9s/step - loss: 0.5839 - accuracy: 0.6795 - val_loss: 0.5521 - val_accuracy: 0.7316
Epoch 2/10
98/98 [==============================] - 945s 10s/step - loss: 0.4328 - accuracy: 0.8146 - val_loss: 0.4905 - val_accuracy: 0.7986
Epoch 3/10
98/98 [==============================] - 965s 10s/step - loss: 0.4376 - accuracy: 0.8058 - val_loss: 0.6977 - val_accuracy: 0.5000
Epoch 4/10
98/98 [==============================] - 961s 10s/step - loss: 0.6897 - accuracy: 0.5395 - val_loss: 0.6815 - val_accuracy: 0.5613
Epoch 5/10
98/98 [==============================] - 977s 10s/step - loss: 0.5079 - accuracy: 0.7634 - val_loss: 0.5164 - val_accuracy: 0.7610
Epoch 6/10
98/98 [==============================] - 956s 10s/step - loss: 0.4349 - accuracy: 0.8186 - val_loss: 0.5526 - val_accuracy: 0.7376
Epoch 7/10
98/98 [==============================] - 950s 10s/step - loss: 0.3593 - accuracy: 0.8581 - val_loss: 0.4214 - val_accuracy: 0.8134
Epoch 8